In [1]:
# Author: Brent Artuch
# Date: 2024-10-26

import sympy as sp
from sympy.physics.quantum import *
import numpy as np
from IPython.display import *

# Qiskit and Q# libraries
import qsharp
from qiskit.quantum_info import Operator, Statevector
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

sp.init_printing()

## Fourier Series
A fourier series is defined as an expansion of a function in a series of sines and cosines dentoted:
\begin{align*}
f(x)=\frac{1}{2}a_0+\sum_{n=1}^{\infty}a_n\cos (nx)+\sum_{n=1}^{\infty}b_n\sin (nx)
\end{align*}

The coefficients $a_0$, $a_n$, and $b_n$ are related to $f(x)$ by definite integrals:
\begin{align*}
\frac{1}{\pi}\int_{0}^{2\pi}f(s)\cos (ns) \ ds=
\begin{cases} 
      2a_0 & n=0 \\
      a_n & n\not=0 \\
   \end{cases}
\end{align*}
Notice that $a_0$ has $2$ as a coefficient which is why we need to multiply $a_0$ by $\frac{1}{2}$ to cancel out that coefficient in the series so that this relationship works for all $n\ge 0$

\begin{align*}
b_n=\frac{1}{\pi}\int_{0}^{2\pi}f(s)\sin (ns) \ ds
\end{align*}
for $n\ge1$.

The exponential form is:
\begin{align*}
f(x)=\sum_{n=-\infty}^{\infty}c_ne^{inx}
\end{align*}
where:
\begin{align*}
c_n=\frac{1}{2}(a_n-ib_n) \ \ \ , \ \ \ c_{-n}=\frac{1}{2}(a_n+ib_n) \ \ \ , \ \ \ n>0 \ \ \ , \ \ \ c_0=\frac{1}{2}a_0 
\end{align*}

If the function $f(x)$ is either an even or an odd function, then the Fourier expansion cannot contain any terms of the opposite parity. So:
\begin{align*}
f(x)=\frac{1}{2}a_0+\sum_{n=1}^{\infty}a_n\cos (nx)
\end{align*}
if the function is $\mathbf{even}$. Conversely:
\begin{align*}
f(x)=\sum_{n=1}^{\infty}b_n\sin (nx)
\end{align*}
if the function is $\mathbf{odd}$.